# How to use OpenAI function calling to query APIs

This tutorial takes you through an example of using OpenAI's new function calling feature to call an external API, such as weather forecasts to NBA statistics. Upon completion, you will be prepared to take on more production-ready scenarios of deploying functions with your own knowledge bases, databases, and other services.

### Getting started

Some basic knowledge of Python and GitHub is helpful for this tutorial. Before diving in, make sure to set up an OpenAI API key and walk through the quickstart tutorial. This will give a good intuition on how to use the API to its full potential.

Python is used as the main programming language along with the OpenAI API. If you run into any issues working through this tutorial, please ask a question on the OpenAI Community Forum.

To start with the code, clone the full code for this tutorial on GitHub. Alternatively, follow along and copy each section into a Jupyter notebook and run the code step by step, or just read along. A good way to avoid any issues is to set up a new virtual environment and install the required packages by running the following commands:

In [44]:
import json
import openai
import os
from termcolor import colored


In [45]:
openai.api_key = os.getenv("OPENAI_API_KEY")

GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"


## Defining functions

An OpenAI ```function``` is defined with a ```name``` that the model will identify it with, a ```description``` that describes when to use it, and ```parameters``` that define what information the LLM needs to gather to use the function. The ```parameters``` can be required or optional, and you can also force the LLM to call a function (or not to call one) should your application logic demand it - for more details on these please refer to [this cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb) and our function calling [docs](https://platform.openai.com/docs/guides/gpt/function-calling).

The LLM should then check the user's message each time to see whether a ```function``` is necessary, and if it does it will return a response that includes the ```finish_reason``` of ```function_call```, triggering your application to call the named function. 

For our example, we'll create two functions, ```get_current_weather``` and ```get_n_day_weather_forecast```. This will not connect with a real API, but will illustrate how functions work and how you can control them.

In [41]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
    {
        "name": "get_n_day_weather_forecast",
        "description": "Get an N-day weather forecast",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
                "num_days": {
                    "type": "integer",
                    "description": "The number of days to forecast",
                }
            },
            "required": ["location", "format", "num_days"]
        },
    },
]

## Calling the OpenAI API with functions

We'll begin by asking it for the weather and seeing how the LLM chooses to react. We expect it to identify that we want to call one of the functions, and prompting us for any additional info it needs.

In [53]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "What's the weather like today"})
chat_response = openai.ChatCompletion.create(
    model=GPT_MODEL,messages=messages, functions=functions
)
assistant_message = chat_response["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

<OpenAIObject at 0x16a410350> JSON: {
  "content": "Sure, I can help you with that. Could you please provide me with your location so that I can fetch the current weather for you?",
  "role": "assistant"
}

We'll clarify our location so it can continue.

In [54]:
messages.append({"role": "user", "content": "I'm in Glasgow, Scotland."})
chat_response = openai.ChatCompletion.create(
    model=GPT_MODEL,messages=messages, functions=functions
)
assistant_message = chat_response["choices"][0]
messages.append(assistant_message)
assistant_message

<OpenAIObject at 0x16a4ddd30> JSON: {
  "finish_reason": "function_call",
  "index": 0,
  "message": {
    "content": null,
    "function_call": {
      "arguments": "{\n  \"location\": \"Glasgow, Scotland\",\n  \"format\": \"celsius\"\n}",
      "name": "get_current_weather"
    },
    "role": "assistant"
  }
}

Great, we've now got a completed function_call. We can tell this by the ```finish_reason``` which is set to ```function_call```. The ```name``` of the function to be called and the ```arguments``` to be passed in to that function are in the ```function_call``` object to be extracted.

We'll now prompt it with a number of days mentioned to try to get it to the target our second function.

In [49]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "what is the weather going to be like in Glasgow, Scotland over the next x days"})
chat_response = openai.ChatCompletion.create(
    model=GPT_MODEL,messages=messages, functions=functions
)
assistant_message = chat_response["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

<OpenAIObject at 0x16a410410> JSON: {
  "content": "Sure! Please provide me with the number of days you would like to know the weather forecast for.",
  "role": "assistant"
}

It should have prompted us for the days, which we'll supply to complete the function call.

In [51]:
messages.append({"role": "user", "content": "5 days"})
chat_response = openai.ChatCompletion.create(
    model=GPT_MODEL,messages=messages, functions=functions
)
chat_response["choices"][0]

<OpenAIObject at 0x16a410710> JSON: {
  "finish_reason": "function_call",
  "index": 0,
  "message": {
    "content": null,
    "function_call": {
      "arguments": "{\n  \"location\": \"Glasgow, Scotland\",\n  \"format\": \"celsius\",\n  \"num_days\": 5\n}",
      "name": "get_n_day_weather_forecast"
    },
    "role": "assistant"
  }
}

Another completed function call - we've now successfully prompted our LLM to call both of the functions we provided. However, this required careful input by the user - in some cases there will be clear cases that we'll want to call a function deterministically, or bar the LLM from calling functions.

For both of these cases, we use the ```function_call``` parameter for ```ChatCompletion``` to get the outcome we want.

## Forcing the use of specific functions or no function

We can force the model to use a specific function, for example ```get_n_day_weather_forecast``` by using the ```function_call``` argument. By doing so, we force the model to make assumptions about how to use it.

In [56]:
# in this cell we force the model to use get_n_day_weather_forecast
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = openai.ChatCompletion.create(
    model=GPT_MODEL,messages=messages, functions=functions,function_call={"name": "get_n_day_weather_forecast"}
)
assistant_message = chat_response["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

<OpenAIObject at 0x16a4ddf10> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"location\": \"Toronto, Canada\",\n  \"format\": \"celsius\",\n  \"num_days\": 1\n}",
    "name": "get_n_day_weather_forecast"
  },
  "role": "assistant"
}

In [57]:
# If we don't force it, it may call the other function
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me a weather report for Toronto, Canada."})
chat_response = openai.ChatCompletion.create(
    model=GPT_MODEL,messages=messages, functions=functions
)
assistant_message = chat_response["choices"][0]["message"]
messages.append(assistant_message)
assistant_message

<OpenAIObject at 0x16a4ddbb0> JSON: {
  "content": null,
  "function_call": {
    "arguments": "{\n  \"location\": \"Toronto, Canada\",\n  \"format\": \"celsius\"\n}",
    "name": "get_current_weather"
  },
  "role": "assistant"
}

We can also force the model to not use a function at all. By doing so we prevent it from producing a proper function call.

In [62]:
messages = []
messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the current weather (use Celcius) for Toronto, Canada."})
chat_response = openai.ChatCompletion.create(
    model=GPT_MODEL,messages=messages, functions=functions,function_call='none',
)
assistant_message = chat_response["choices"][0]["message"]
assistant_message

<OpenAIObject at 0x1089b31d0> JSON: {
  "content": "{\n  \"location\": \"Toronto, Canada\",\n  \"format\": \"celsius\"\n}",
  "role": "assistant"
}

## Calling an external function

In most cases when using function calling you'll want to take the ```arguments``` and plug them into an API to get some result. We'll add in an external API to test this flow out end-to-end. It will work like this:
- Our user will ask the LLM a question.
- The LLM will prompt the user for the required information.
- Once the LLM has what it needs, it will output a function call.
- We'll take the function call and use it to call an API to get some results.
- We'll hand those results back to the LLM, who will use them to answer the user.

Lets test it out!

In [63]:
nba_functions = [
    {
        "name": "get_stats",
        "description": "Get NBA stats for a player or team.",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {
                    "type": "string",
                    "description": "Name of the team or player that we want to get stats for.",
                },
                "type": {
                    "type": "string",
                    "enum": ["player", "team"],
                    "description": "Whether the stats are for a player or team. Infer this from the user's question.",
                },
            },
            "required": ["name", "type"],
        },
    }
]

We'll add a simple function to print the conversation we create with the various roles highlighted so we can audit the back-and-forth between ```user``` and ```assistant```, supported by the ```system``` message to give instruction and the ```function``` results which give response for the LLM to act on.

In [64]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    formatted_messages = []
    for message in messages:
        if message["role"] == "system":
            formatted_messages.append(f"system: {message['content']}\n")
        elif message["role"] == "user":
            formatted_messages.append(f"user: {message['content']}\n")
        elif message["role"] == "assistant" and message.get("function_call"):
            formatted_messages.append(f"assistant: {message['function_call']}\n")
        elif message["role"] == "assistant" and not message.get("function_call"):
            formatted_messages.append(f"assistant: {message['content']}\n")
        elif message["role"] == "function":
            formatted_messages.append(f"function ({message['name']}): {message['content']}\n")
    for formatted_message in formatted_messages:
        print(
            colored(
                formatted_message,
                role_to_color[messages[formatted_messages.index(formatted_message)]["role"]],
            )
        )

In [97]:
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import commonplayerinfo, teamplayerdashboard


def get_player_stats(player_name):
    
    # Find players by first name.
    player_id = players.find_players_by_full_name('durant')[0]['id']
    
    # Get common player info for player
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
    
    player_df = player_info.player_headline_stats.get_data_frame()
    
    return player_df

def get_team_stats(team_name):
    
    # Find team by name.
    team_id = teams.find_teams_by_full_name('cav')[0]['id']
    
    # Get info for the team
    team_info = teamplayerdashboard.TeamPlayerDashboard(team_id=team_id)
    
    team_df = team_info.team_overall.get_data_frame()
    
    return team_df

In [98]:
def execute_function_call(message):
    if message["function_call"]["name"] == "get_stats":
        
        if json.loads(message["function_call"]["arguments"])["type"] == 'player':
            player_name = json.loads(message["function_call"]["arguments"])["name"]
            results = get_player_stats(player_name)
            
        else:
            
            team_name = json.loads(message["function_call"]["arguments"])["name"]
            results = get_team_stats(team_name)
    else:
        results = f"Error: function {message['function_call']['name']} does not exist"
    return results

We'll now initiate a fresh conversation and ask some NBA related questions, checking that the LLM reacts as expected and fetches us the information we requested.

In [102]:
messages = []
messages.append({"role": "system", "content": "Answer user questions with the stats from your functions. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "Give me the points and rebounds for Kevin Durant"})
chat_response = openai.ChatCompletion.create(
    model=GPT_MODEL,messages=messages, functions=nba_functions
)
assistant_message = chat_response["choices"][0]["message"]
messages.append(assistant_message)
if chat_response['choices'][0]['finish_reason'] == 'function_call':
    print('Making function call')
    assistant_message = chat_response["choices"][0]["message"]
    function_response = execute_function_call(assistant_message)
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": str(function_response)})
    chat_response = openai.ChatCompletion.create(
    model=GPT_MODEL,messages=messages, functions=nba_functions,function_call='none'
    )
    assistant_message = chat_response["choices"][0]["message"]
else:
    assistant_message = chat_response["choices"][0]["message"]
messages.append(assistant_message)
assistant_message['content']

Making function call


'Kevin Durant has an average of 29.1 points and 6.7 rebounds per game.'

In [103]:
pretty_print_conversation(messages)

system: Answer user questions with the stats from your functions. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: Give me the points and rebounds for Kevin Durant

assistant: {
  "arguments": "{\n  \"name\": \"Kevin Durant\",\n  \"type\": \"player\"\n}",
  "name": "get_stats"
}

function (get_stats):    PLAYER_ID   PLAYER_NAME TimeFrame   PTS  AST  REB    PIE
0     201142  Kevin Durant   2022-23  29.1  5.0  6.7  0.185

assistant: Kevin Durant has an average of 29.1 points and 6.7 rebounds per game.



In [104]:
messages = []
messages.append({"role": "system", "content": "Answer user questions with the stats from your functions. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
messages.append({"role": "user", "content": "How did the Cavaliers do last season"})
chat_response = openai.ChatCompletion.create(
    model=GPT_MODEL,messages=messages, functions=nba_functions
)
assistant_message = chat_response["choices"][0]["message"]
messages.append(assistant_message)
if chat_response['choices'][0]['finish_reason'] == 'function_call':
    print('Making function call')
    assistant_message = chat_response["choices"][0]["message"]
    function_response = execute_function_call(assistant_message)
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": str(function_response)})
    chat_response = openai.ChatCompletion.create(
    model=GPT_MODEL,messages=messages, functions=nba_functions,function_call='none'
    )
    #print(chat_response)
    assistant_message = chat_response["choices"][0]["message"]
else:
    assistant_message = chat_response["choices"][0]["message"]
messages.append(assistant_message)
assistant_message['content']

Making function call


"The Cleveland Cavaliers had a record of 51 wins and 31 losses last season, with a winning percentage of 0.622. They played a total of 82 games. The team's overall performance was ranked first in several statistical categories such as rebounds, assists, turnovers, steals, blocks, blocks against, personal fouls, free throw attempts, and points scored."

In [105]:
pretty_print_conversation(messages)

system: Answer user questions with the stats from your functions. Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: How did the Cavaliers do last season

assistant: {
  "arguments": "{\n  \"name\": \"Cavaliers\",\n  \"type\": \"team\"\n}",
  "name": "get_stats"
}

function (get_stats):   GROUP_SET     TEAM_ID            TEAM_NAME GROUP_VALUE  GP   W   L  W_PCT   
0   Overall  1610612739  Cleveland Cavaliers     2022-23  82  51  31  0.622  \

      MIN   FGM  ...  REB_RANK  AST_RANK  TOV_RANK  STL_RANK  BLK_RANK   
0  3976.0  3408  ...         1         1         1         1         1  \

   BLKA_RANK  PF_RANK  PFD_RANK  PTS_RANK  PLUS_MINUS_RANK  
0          1        1         1         1                1  

[1 rows x 56 columns]

assistant: The Cleveland Cavaliers had a record of 51 wins and 31 losses last season, with a winning percentage of 0.622. They played a total of 82 games. The team's overall performan

## Conclusion

If the LLM is inconsistent in calling the functions or supplying the arguments you want, you should dedicate time to engineering both the function descriptions and the system prompt as these have a huge bearing on the output. If you are using multiple functions, you should also take care to ensure the case for using each is clear and that there is not significant overlap between them - if this happens, you will have an uneven user experience where the LLM sometimes calls one function and sometimes calls another.

For production solutions you should also consider using a more permanent storage and retrieval solution such as a vector database to minimize cost and latency as you scale. There are details on the sorts of solutions available on our [documentation](https://platform.openai.com/docs/guides/embeddings/how-can-i-retrieve-k-nearest-embedding-vectors-quickly) and in the [cookbook](https://github.com/openai/openai-cookbook/tree/main/examples/vector_databases).

We hope you've enjoyed this introduction to function calling and long document summarization, and we look forward to seeing what you build!